In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
import json
import logging
import os
import re
import sys
from functools import partial
from itertools import chain
from pathlib import Path
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display
from pandas.tseries.offsets import BDay
from scipy.interpolate import CubicSpline
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser

from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)
pd.options.plotting.backend = "plotly"

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-02-01 07:53:28,394 - Authenticating...
2023-02-01 07:53:29,165 - Successfully authenticated


In [5]:
future_months_codes = [
    'F',
    'G',
    'H',
    'J',
    'K',
    'M',
    'N',
    'Q',
    'U',
    'V',
    'X',
    'Z'
]

future_order = [f"{month}{year}" for year in range(0, 12 + 1) for month in future_months_codes]

In [6]:
mid_curve_map = {
    'sfr': [
        # 'sra', 
        # 'srr', 
        # 'srw', 
        '0q', 
        '2q', 
        '3q',
        '4q',
    ],
    'er': [
        '0r', 
        '2r', 
        '3r',
        '4r'
    ]
}

fixing_map = {
    'sfr': 'SOFRRATE Index',
    'er': 'EUR003M Index'
}

In [7]:
def get_underlying_ticker(underlying, contract, yk=None):
    yk = yk or 'Comdty'
    return f"{underlying.upper()}{contract.upper() if isinstance(contract, str) else contract} {yk.title()}"

In [8]:
def get_fut_chain(underlying, contracts=None, yk=None):
    contracts = contracts or range(1, 16 + 1)
    yk = yk or 'Comdty'
    securities = [get_underlying_ticker(underlying, contract, yk) for contract in contracts]
    tickers = [*(bq.bdp(securities, {"FUT_CUR_GEN_TICKER": 'ticker'})['ticker'] + ' ' + yk)]
    ticker_map = {contract: ticker for contract, ticker in zip(contracts, tickers)}
    prices = bq.bdp([fixing_map[underlying], *tickers], {"px_last": 'price', "LAST_TRADEABLE_DT": 'expiry'})
    prices.loc[prices['security'] == fixing_map[underlying], 'expiry'] = today()
    prices.loc[prices['security'] == fixing_map[underlying], 'price'] = 100 - prices.loc[prices['security'] == fixing_map[underlying], 'price']
    days = (prices['expiry'] - today()).dt.days
    prices['days'] = days
    return prices

In [9]:
def get_opt_chain_and_info(underlying):
    chain_ = bq.bds(underlying, 'OPT_CHAIN')
    if chain_.empty:
        return
    chain_ = chain_.squeeze().rename('chain')
    chain = chain_.str.replace(r'\s+', ' ', regex=True).str.split(' ')
    chain = pd.DataFrame.from_dict(dict(zip(chain.index, chain.values))).T.set_axis(['code', 'strike', 'yk'], axis=1)
    chain['call_put'] = chain['code'].str[-1]
    chain['code'] = chain['code'].str[:-1]
    chain['strike'] = chain['strike'].astype(float)
    chain['orig'] = chain_
    
    opt_chain = chain.drop_duplicates('code').set_index('code')['orig']
    opt_expiry = bq.bdp(opt_chain, {'LAST_TRADEABLE_DT': 'expiry'}).rename({'security': 'orig'}, axis=1)
    
    def get_strike_increment(df):
        return df['strike'].head(2).diff().dropna().squeeze()
    increments = chain.groupby('code', sort=False).apply(get_strike_increment).rename('inc')
    opt_info = opt_expiry.merge(opt_chain.reset_index(), on='orig').merge(increments, on='code')
    
    return chain, opt_info

In [10]:
def get_fut_and_opt_info(underlying, contract, yk=None, interp=None):
    prices = get_fut_chain(underlying, yk=yk)
    interp = interp or CubicSpline
    spline = interp(prices['days'], prices['price'])
    res_q = get_opt_chain_and_info(get_underlying_ticker(underlying, contract, yk))
    if not res_q:
        return
    mid_curves_undl = [get_underlying_ticker(tic, contract, yk) for tic in mid_curve_map[underlying.lower()]]
    
    mid_crv_ress = []                   
    for tic in mid_curves_undl:
        res_mid_curve = get_opt_chain_and_info(tic)
        if res_mid_curve:
            mid_crv_ress.append(res_mid_curve)
    
    if not mid_crv_ress:
        opt_chain = res_q[0]
        opt_info = res_q[1]
    else:
        opt_chain = pd.concat([res_q[0], *[res_mid_curve[0] for res_mid_curve in mid_crv_ress]])
        opt_info = pd.concat([res_q[1], *[res_mid_curve[1] for res_mid_curve in mid_crv_ress]])
        
    price = prices.set_index('security')['price'].loc[get_underlying_ticker(underlying, contract, yk)]
    fut_exp = prices.set_index('security')['expiry'].loc[get_underlying_ticker(underlying, contract, yk)]
    opt_prices = {}
    
    for index, group in opt_chain.groupby('code', sort=False):
        closest_stike_idx = (group['strike'] - price).abs().idxmin()
        closest_stike = group.loc[closest_stike_idx]['strike']
        atm_opts = group.query(f"strike == {closest_stike}")
        opt_px = bq.bdp(atm_opts['orig'], {"px_last": "price"})
        opt_exp = opt_info.set_index('code').loc[index, 'expiry']
        #         
        interp_x = (opt_exp - today()).days
        interp_px = spline(interp_x).item()         
        roll = interp_px - price
        #         
        interp_x = (fut_exp - opt_exp).days
        interp_px = spline(interp_x).item()         
        roll = interp_px - price
        #        
        
        straddle = opt_px['price'].sum()
        opt_prices[index] = opt_px
        # res_ = opt_px.assign(code=index).pivot(index='code', columns='security', values='price').reset_index().rename_axis(None, axis=1)
        res_ = opt_px.melt().assign(sec=['P', 'C']*2)
        res_['sec'] = res_['variable'] + '_' + res_['sec']
        res_ = res_.set_index('sec')['value'].to_frame().T.rename_axis(None, axis=1)
        res_ = res_.assign(
            opt_exp=opt_exp,
            fut_exp=fut_exp,
            fut_price=price,
            fut_price_aged=interp_px,
            roll=roll,
            atm_straddle=straddle,
            ratio=roll/straddle,
            abs_ratio=np.abs(roll/straddle)
        )
        opt_prices[index] = res_
    res = pd.concat(opt_prices)
    res = pd.concat([res.droplevel(1)], keys=[get_underlying_ticker(underlying, contract, yk)]).rename_axis(['underlying', 'opt_code'])
    return res, prices, opt_chain, opt_info

In [11]:
contract_numbers = range(1, 16 + 1)
contracts = ['h3', 'm3', 'u3', 'z3', 'h4', 'm4', 'u4', 'z4', 'h5', 'm5', 'u5', 'z5']
underlyings = ['sfr', 'er']
yk = 'Comdty'

In [12]:
ress_all = {}
df_all = {}
mat_all = {}

In [13]:
for underlying in underlyings:
    ress = [] 
    for contract in contracts:
        res = get_fut_and_opt_info(underlying, contract)
        if res:
            ress.append(res)
    ress_all[underlying] = ress
    df = pd.concat([res[0] for res in ress])
    df['opt_exp_code'] = pd.Categorical(df.index.get_level_values(1).str[-2:], ordered=True, categories=future_order)
    df['fut_exp_code'] = pd.Categorical(df.index.get_level_values(0).str.split(' ').str[0].str[-2:], ordered=True, categories=future_order)
    mat = df.pivot(index='fut_exp_code', columns='opt_exp_code', values='abs_ratio')
    df_all[underlying] = df
    mat_all[underlying] = mat

In [14]:
import xlwings as xw
from pathlib import Path
from win32com import client as win32

In [15]:
wb_name = 'option_roll.xlsx'
writer = pd.ExcelWriter(wb_name, engine='xlsxwriter', date_format="ddmmmyy")
for i, underlying in enumerate(underlyings):
    # if i:
    #     wb.sheets.add(name=f'{underlying}')
    #     wb.sheets.add(name=f'{underlying} details')
    # else:
    #     wb.sheets[0].name = f'{underlying}'
    #     # wb.sheets.add(name=f'{underlying} details')
        
    mat, df = mat_all[underlying], df_all[underlying]
    # wb.sheets[f'{underlying}'].range('A1').value = mat
    m = mat.rename_axis(None, axis=1).rename_axis(None)
    m = pd.concat([pd.concat([m], axis=1, keys=['option expiry'])], keys=['future expiry'])
    m.to_excel(writer, sheet_name=f'{underlying}', float_format='%.2f')
    sh = writer.sheets[f'{underlying}']
    cell_format = writer.book.add_format({'valign': 'vcenter', 'bold': True, 'border': 1})
    sh.write(3,  0, 'future expiry', cell_format)
    sh.conditional_format(2, 2, 2 + m.shape[0],  2 + m.shape[1], {'type': '3_color_scale'})
    sh.autofit()
    df.to_excel(writer, sheet_name=f'{underlying} details')
    sh2 = writer.sheets[f'{underlying} details']
    sh2.autofit()
writer.close()

In [16]:
attachment_path = str(Path().resolve() / wb_name)

In [17]:
wb = xw.Book(attachment_path)
for sh in wb.sheets:
    sh.autofit()
wb.save(attachment_path)
wb.close()

In [18]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'giovambattista.perciaccante@brevanhoward.com; menashe.banit@brevanhoward.com'
# mail.To = 'giovambattista.perciaccante@brevanhoward.com'
mail.Subject = f'Option roll as of {today(): %d%b%y}'
# mail.Body = 'Message body'
# mail.HTMLBody = html  # this field is optional
# To attach a file to the email (optional):
attachment  = attachment_path
mail.Attachments.Add(attachment)
mail.Send()